In [214]:
# Import packages
 # import fuzzywizzy?
import pandas as pd
import numpy as np
import pymongo
import string
import nltk
import re
import os
import sys
import pickle


src_dir = os.path.join(os.getcwd(), '..', '..', 'src')
sys.path.append(src_dir)

# helper functions
from d02_processing.cleaning_signatures import clean_signatures

# Load the "autoreload" extension
%load_ext autoreload

# reload modules so that as you change code in src, it gets loaded
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# open up a Mongo Connection and create a cursor
myclient = pymongo.MongoClient("mongodb://127.0.0.1:27017/")
curlydb = myclient['Naturaly_Curly_db']
curly_collection = curlydb['curly_profiles']

In [4]:
# Access all the unique items and store them to a list
query_1 = curly_collection.find({})
test = []
for x in query_1:
    test.append(x['signature'])

# This is the number of total entire in the database
print(len(test)) 

532489


In [195]:
# Create a set of unique items in the db and inspect it
unique_sigs = list(set(test))
print(type(unique_sigs))
print(len(unique_sigs))
print(unique_sigs[5000:5003])

<class 'list'>
7961
["3b curls, big chop Nov 07\n\nCurrent fave products: Aubrey Organics Honeysuckle Rose Shampoo and Conditioner, Aveda Curl Enhancer, Aveda Flaxseed Aloe gel\n\nStill like: Ouidad CC gel\nDon't like: Deva ArcAngell", '2b waves\nMedium texture, high porosity\nBronde & shoulder length, trying to grow it past my shoulder blades.\nWent natural on June 4th, 2010\nMy hair is hungry for PROTEIN!! :munky2:', '-3A/Medium/Low porosity\n\n-Cg since 01/2012\n-Growing out since 11/2011\n\n-Co-wash: Suave Naturals']


In [216]:
filename = '../../src/d01_utils/signature_list'
outfile = open(unique_sigs,'wb')
pickle.dump(unique_sigs, outfile)
outfile.close()

TypeError: expected str, bytes or os.PathLike object, not list

In [196]:
# test a cleaning function on a small sample size
samples = unique_sigs[0:50]
samples

['',
 'Lions, Tigers and My Blog',
 "Very modified CG. Activate and B2B pomegranate Condish. Re:coil, Joiwhip, LOOB, HE set me up and orange bottle. Boots, cant' remember anymore.\nIn a place where dry and burned straight hair is so common my wild spanish curls are my greatest weapon!",
 'Mod CG since 06/08/2011-trying to figure it all out!\n\nHair type: 2b/2c, medium texture, high (??) porosity\nCo-Wash: Tresemme Naturals\nRO: AOHR\nLI: Tresemme Naturals\nStylers: Jessicurl RR + CC, AG Re:Coil (still trying to love this), and Curl Keeper\nGels: EcoStyler, BRHG, Got2B Spiked Up\nTried: Aloe (ok for frizz), FSG (so wanted this to work)',
 '3a/b.',
 "\nFine, curly/wavy, prone to breakage. Still figuring out curl pattern...3A on some days (only w/ product), 2C on others (but not coarse). Would love help on hair-typing! Low porosity (I think). I thought it was low-density, but a hairstylist told me I have tiny hairs growing in underneath that break, so that's why it's thin). Product junkie

In [39]:
lower_sample=[]
for sample in samples:
    lower_sample.append(sample.lower().split())
lower_sample


[[],
 ['lions,', 'tigers', 'and', 'my', 'blog'],
 ['very',
  'modified',
  'cg.',
  'activate',
  'and',
  'b2b',
  'pomegranate',
  'condish.',
  're:coil,',
  'joiwhip,',
  'loob,',
  'he',
  'set',
  'me',
  'up',
  'and',
  'orange',
  'bottle.',
  'boots,',
  "cant'",
  'remember',
  'anymore.',
  'in',
  'a',
  'place',
  'where',
  'dry',
  'and',
  'burned',
  'straight',
  'hair',
  'is',
  'so',
  'common',
  'my',
  'wild',
  'spanish',
  'curls',
  'are',
  'my',
  'greatest',
  'weapon!'],
 ['mod',
  'cg',
  'since',
  '06/08/2011-trying',
  'to',
  'figure',
  'it',
  'all',
  'out!',
  'hair',
  'type:',
  '2b/2c,',
  'medium',
  'texture,',
  'high',
  '(??)',
  'porosity',
  'co-wash:',
  'tresemme',
  'naturals',
  'ro:',
  'aohr',
  'li:',
  'tresemme',
  'naturals',
  'stylers:',
  'jessicurl',
  'rr',
  '+',
  'cc,',
  'ag',
  're:coil',
  '(still',
  'trying',
  'to',
  'love',
  'this),',
  'and',
  'curl',
  'keeper',
  'gels:',
  'ecostyler,',
  'brhg,',
  'got

In [1]:
tokens = word_tokenize(samples[33])
tokens

NameError: name 'word_tokenize' is not defined

In [49]:
def generate_ngrams(s, n):
    # Convert to lowercases
    s = s.lower()
    
    # Replace all none alphanumeric characters with spaces
    s = re.sub(r'[^a-zA-Z0-9\s]', ' ', s)
    
    # Break sentence in the token, remove empty tokens
    tokens = [token for token in s.split(" ") if token != ""]
    
    # Use the zip function to help us generate n-grams
    # Concatentate the tokens into ngrams and return
    ngrams = zip(*[tokens[i:] for i in range(n)])
    return [" ".join(ngram) for ngram in ngrams]

In [50]:
generate_ngrams(samples[33], 2)

['2b 3a',
 '3a primarily',
 'primarily use',
 'use jessicurl',
 'jessicurl and',
 'and devacurl',
 'devacurl products',
 'products curls',
 'curls hair',
 'hair tea',
 'tea conditioner',
 'conditioner and',
 'and various',
 'various gels',
 'gels i',
 'i m',
 'm modified',
 'modified cg',
 'cg ish',
 'ish since',
 'since 5',
 '5 04']

In [53]:
s = re.sub(r'[^a-zA-Z0-9\s]', ' ', samples[33])
tokens = word_tokenize(s)
vocabulary = set(tokens)
print(len(vocabulary))
frequency_dist = nltk.FreqDist(tokens)
sorted(frequency_dist,key=frequency_dist.__getitem__, reverse=True)[0:50]


22


['and',
 '2b',
 '3a',
 'primarily',
 'use',
 'Jessicurl',
 'DevaCurl',
 'products',
 'Curls',
 'Hair',
 'Tea',
 'conditioner',
 'various',
 'gels',
 'I',
 'm',
 'modified',
 'CG',
 'ish',
 'since',
 '5',
 '04']

In [66]:
s = re.sub(r'[^a-zA-Z0-9\s]', ' ', samples[34])
s = s.lower()
s

'mostly 3a  some 3b 2c fine med texture med porosity low density\n\nwash  alternate co washing with v05   soap bars\nrinse out  rotate tj s n s  aohr  lvpng  gvp k pak  cj ca daily condish\nacv rinses 2x wk  high ph water \nli  if i use one  it s usually cj cali cj h b li  or gdli\nstyling  fsg with avg  an  hemp seed oil   slippery elm  winter only   kccc  cj cccc  cj sg  cj afg  aomm   brhg \n\ncurrently experimenting w dm miracurls cg   mm'

In [63]:
samples[34]

"Mostly 3A, some 3B/2C fine-med texture/med porosity/low density\n\nwash: alternate co-washing with V05 & soap bars\nRinse out: Rotate TJ's N&S, AOHR, LVPNG, GVP K-Pak, CJ CA Daily Condish\nACV rinses 2x/wk (high pH water)\nLI: if I use one, it's usually CJ CALI CJ H&B LI, or GDLI\nStyling: FSG with AVG, AN, hemp seed oil & slippery elm (winter only), KCCC, CJ CCCC, CJ SG, CJ AFG, AOMM & BRHG \n\nCurrently experimenting w/DM Miracurls CG & MM"

In [211]:
# Clean up the signatures
# Take the four characteristics and put them in lists (put absent ones as null values) while removing them
# remove www, http, cg, cgm completly
# put the remainder into a string of no puntation and lowercase(inspect these and maybe apply stop words...maybe)
# Finally read the string 

curls = []
density = []
porosity = []
texture = []
products = []

# maybe make a seperate clean up function fromthe make a data frame function with all of this substitutions and different product names
s = samples[47]
s = re.sub(r'[^a-zA-Z0-9\s]', ' ', s)
s = s.lower()
s = re.sub(r"\n", " ", s)
s = re.sub(r"med\s", "medium ", s)
s = re.sub(r"hi\s", "high ", s)
s = re.sub(r"po\s", "porosity ", s)
s = re.sub(r"den\s", "density ", s)
# lot of hair --> thick density
print(s)
################## 
### Curl shape
curl_pattern = r"\s\da|\s\db|\s\dc"
curl_result1 = re.search(curl_pattern, s)

#First 
try:
    s = s.replace(curl_result1.group(), "")
    curls.append(curl_result1.group())
except AttributeError:
    #if error put np.nan into list
    curls.append(np.nan)

curl_result2 = re.search(curl_pattern, s)
try:
    s = s.replace(curl_result2.group(), "")
    curls.append(curl_result2.group())
    two_shapes = True
except AttributeError:
    #if error put np.nan into list for first instance and set a variable for other characteristics to know if they need to put two row  
    two_shapes = False

s = re.sub("  ", " ",s)
s = re.sub("   ", " ",s)

##################    
### Texture Patten
# Check all the conditions that might be present to describe the texture including if they don't follow it with the word texture
texture_pattern1 = r"texture"
texture_result1 = re.search(texture_pattern1, s)

texture_pattern2 = r"course"
course_result = re.search(texture_pattern2, s)

texture_pattern3 = r"fine"
fine_result = re.search(texture_pattern3, s)

texture_pattern4 = r"\w*(?= texture)"
texture_result4 = re.search(texture_pattern4, s)

if texture_result1 is not None:
    try:
        s = s.replace(texture_result4.group(), texture_result4.group())
        s = re.sub(r"\w*( texture)", "", s)
        if two_shapes:
            texture.append(texture_result4.group())
            texture.append(texture_result4.group())  
        else:
            texture.append(texture_result4.group())
    except AttributeError:
        if two_shapes:
            texture.append(np.nan)
            texture.append(np.nan)  
        else:
            texture.append(np.nan)

elif course_result is not None:      
    s = s.replace(course_result.group(), "")
    if two_shapes:
        texture.append(course_result.group())
        texture.append(course_result.group())  
    else:
        texture.append(course_result.group())

elif fine_result is not None:      
    s = s.replace(fine_result.group(), "")
    if two_shapes:
        texture.append(fine_result.group())
        texture.append(fine_result.group())  
    else:
        texture.append(fine_result.group())
else:
    if two_shapes:
        texture.append(np.nan)
        texture.append(np.nan)  
    else:
        texture.append(np.nan)

s = re.sub("  ", " ",s)
s = re.sub("   ", " ",s)

################## 
### Porosity Pattern
porosity_pattern = r"\w*(?= porosity)"
porosity_result = re.search(porosity_pattern, s)

try:
    s = s.replace(porosity_result.group(), porosity_result.group())
    s = re.sub(r"\w*( porosity)", "", s)
    if two_shapes:
        porosity.append(porosity_result.group())
        porosity.append(porosity_result.group())  
    else:
        porosity.append(porosity_result.group())
except AttributeError:
    if two_shapes:
        porosity.append(np.nan)
        porosity.append(np.nan)  
    else:
        porosity.append(np.nan)

s = re.sub("  ", " ",s)
s = re.sub("   ", " ",s)

################## 
### Density Pattern
density_pattern1 = r"density"
density_result1 = re.search(density_pattern1, s)

density_pattern2 = r"thin"
thin_result = re.search(density_pattern2, s)

density_pattern3 = r"thick"
thick_result = re.search(density_pattern3, s)

density_pattern4 = r"\w*(?= density)"
density_result4 = re.search(density_pattern4, s)

if density_result1 is not None:
    try:
        s = s.replace(density_result4.group(), density_result4.group())
        s = re.sub(r"\w*( density)", "", s)
        if two_shapes:
            density.append(density_result4.group())
            density.append(density_result4.group())  
        else:
            density.append(density_result4.group())
    except AttributeError:
        if two_shapes:
            density.append(np.nan)
            density.append(np.nan)  
        else:
            density.append(np.nan)

elif thin_result is not None:      
    s = s.replace(thin_result.group(), "")
    if two_shapes:
        density.append(thin_result.group())
        density.append(thin_result.group())  
    else:
        density.append(thin_result.group())

elif thick_result is not None:      
    s = s.replace(thick_result.group(), "")
    if two_shapes:
        density.append(thick_result.group())
        density.append(thick_result.group())  
    else:
        density.append(thick_result.group())

else:
    if two_shapes:
        density.append(np.nan)
        density.append(np.nan)  
    else:
        density.append(np.nan)



#### Maybe go back now and remove third curl pattern references, medium, fine, thick, thin,corse etc since it must have ben two parters        
#s = s.replace()
s = re.sub(r"(http.*?\s)", "", s)
s = re.sub(r"(www.*?\s)", "", s)
s = re.sub(r"(cg.*?\s)", "", s)
s = re.sub(r"\s\da|\s\db|\s\dc", "", s)
s = re.sub("fine", "", s)
s = re.sub("course", "", s)
s = re.sub("thick", "", s)
s = re.sub("thin", "", s)
s = re.sub("medium", "", s)
s = re.sub("  ", " ",s)
s = re.sub("   ", " ",s)

if two_shapes:
    products.append(s) 
    products.append(s) 
else:
    products.append(s)   
    
    
# print(samples[47])
# print(s)
print(curls)
print(texture)
print(porosity)
print(density)
print(products)
print(two_shapes)

3a   dark brown   medium length  curl pride   cat 
[nan]
[nan]
[nan]
[nan]
['3a dark brown length curl pride cat ']
False


In [174]:
# when putting all the lists together into the data frame make sure they are all the same length
print(len(curls))
print(len(density))
print(len(porosity))
print(len(texture))
print(len(products))

60
60
60
60
60


In [190]:
if len(curls) == len(density) == len(porosity) == len(texture) == len(products):
    None
else:
    print('differeing list lengths')

In [175]:
characteristics_dict = {'curl_pattern': curls, 'density': density, 'porosity': porosity, 'texture': texture, 'products': products}

signature_df = pd.DataFrame(characteristics_dict)
signature_df

,curl_pattern,density,porosity,texture,products
0,NaN,NaN,NaN,NaN,
1,NaN,NaN,NaN,NaN,lions tigers and my blog
2,NaN,NaN,NaN,NaN,very modified activate and b2b pomegranate con...
3,2b,NaN,high,medium,mod since 06 08 2011 trying to figure it all o...
4,2c,NaN,high,medium,mod since 06 08 2011 trying to figure it all o...
5,NaN,NaN,NaN,NaN,3a b
6,3a,low,low,fine,curly wavy prone to breakage still figuring o...
7,2c,low,low,fine,curly wavy prone to breakage still figuring o...
8,NaN,NaN,NaN,NaN,scarletrook password curlytop
9,NaN,NaN,NaN,NaN,spunky ducky i love you


In [213]:
df = clean_signatures(samples)
df.tail(30)
    



,curl_pattern,density,porosity,texture,products
32,4b,NaN,NaN,NaN,natural
33,NaN,thin,NaN,NaN,descartes walks into a bar the bartender says ...
34,3b,thick,NaN,NaN,bc 3 05 sl curly 50 curl factor faves fall wi...
35,3c,thick,NaN,NaN,bc 3 05 sl curly 50 curl factor faves fall wi...
36,4a,NaN,NaN,NaN,indigo holding it down for the s with a little...
37,4b,NaN,NaN,NaN,indigo holding it down for the s with a little...
38,3b,high,low,fine,sigpic sigpic kosher curls mostly curls coars...
39,3b,NaN,NaN,NaN,hair type
40,NaN,NaN,NaN,NaN,2nd time natural last relaxer feb 09 bc july a...
41,NaN,NaN,NaN,NaN,sigpic sigpic


In [206]:
samples[47]

'3A | Dark Brown | Medium-Length\n\nCurl pride! :cat:'